# Сортировка предложений

## Загружаем классификатор и словарь:

In [139]:
#!g1.1
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [140]:
#!g1.1
class Clf(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, output_dim):
        
        super().__init__()          
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.flatten = nn.Flatten() # вместо усреднения склеим все в 1 вектор
        self.fc = nn.Linear(embedding_dim*MAX_LEN, output_dim) # размер склееного вектора - размер эмбединга на MAX_LEN
        self.act = nn.Sigmoid() # активацию менять даже не пришлось, так как у нас бинарная классификация
                                # если классов больше 2, то нужно поставить nn.Softmax()
        
    def forward(self, text):
        
        embedded = self.embedding(text)   
        hidden = self.flatten(embedded)
        dense_outputs=self.fc(hidden)
        outputs=self.act(dense_outputs) 
        
        return outputs

In [141]:
#!g1.1
MAX_LEN = 70
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load again the model that we saved
model = Clf(14250, 30, 1)
model.load_state_dict(torch.load("clf_simple.pt"))
model.eval()

Clf(
  (embedding): Embedding(14250, 30)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=2100, out_features=1, bias=True)
  (act): Sigmoid()
)

In [142]:
#!g1.1
filtered_vocab = torch.load('filtered_vocab.pth')

In [143]:
#!g1.1
# индексируем слова
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

## Загружаем предложения для сортировки

In [137]:
#!g1.1
rsl_svo_50 = pd.read_csv("rsl_svo_50.tsv", sep="\t", names=['rsl', 'rus'])
rsl_svo_50.head()

,rsl,rus
0,муха говорить на жестовом языке цвет,Муха говорить на жестовом языке цвет.
1,муха говорить на жестовом языке будущие родители,Муха говорить на жестовом языке будущие родители.
2,муха говорить на жестовом языке миллиард,Муха говорить на жестовом языке миллиард.
3,муха говорить на жестовом языке ноль,Муха говорить на жестовом языке ноль.
4,муха говорить на жестовом языке боль,Муха говорить на жестовом языке боль.


In [ ]:
#!g1.1


In [57]:
#!g1.1
device

device(type='cuda')

In [58]:
#!g1.1
#from tqdm import tqdm


#def write(filename, data):
#    for line in data:
#        with open(filename, 'a', encoding='utf-8') as f:
#            f.write(line[0] + '\t' + line[1] + '\n')
            
            
#with open('good_fake_sentences_rsl_svo_50.tsv', 'w', encoding='utf-8') as f:
#    pass

#model.cuda()
#data_to_save = []
#for i in tqdm(range(len(rsl_svo_50))):  #[:200000]
#    data_list = rsl_svo_50.rus.to_list()
#    sent = prepare_texts_to_pred(data_list, i)
#    single_pred = model(sent.unsqueeze(0).to(device)).detach().to('cpu').numpy().tolist()[0][0]
        
    # вообще лучший f1-score для класса хороших предожений был на 0.4
#    if single_pred > 0.25: 
        # for testing: print(rsl_svo_50.rus.to_list()[i], round(single_pred, 4))
#        data_to_save.append((data_list[i], data_list[i]))
    
#    if i % 100000 == 0:
#        write('good_fake_sentences_rsl_svo_50.tsv', data_to_save)
#        data_to_save = []


In [138]:
#!g1.1
from tqdm import tqdm


def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [token for token in tokens if token]
    return tokens

def prepare_texts_to_pred(text):
    # в index будут индексы от 0 до len(rsl_svo_50.rus) = len(data)
    # по ним мы достанем тексты, предобработаем, переведем в векторы, западим и вернем
    tokens = preprocess(text) # токенизируем
    ids = [word2id[token] for token in tokens if token in word2id][:MAX_LEN]
    ids = torch.nn.functional.pad(torch.LongTensor(ids), 
                                  (0, MAX_LEN-len(ids)), 
                                  mode='constant',
                                  value=0)
    return ids

model.to(device)

with open('good_fake_sentences_rsl_svo_50.tsv', 'w', encoding='utf-8') as f:
    for row in tqdm(rsl_svo_50.itertuples()):  #[:200000]
        rus = row.rus
        rsl = row.rsl
        
        sent = prepare_texts_to_pred(rus)
        # single_pred = model(sent.unsqueeze(0).to(device)).detach().to('cpu').numpy().tolist()[0][0]
        single_pred = model(sent.unsqueeze(0).to(device)).detach().to('cpu').numpy().tolist()[0][0]
        
        # вообще лучший f1-score для класса хороших предожений был на 0.4
        if single_pred > 0.: 
            # print(rus, round(single_pred, 4))
            f.write(rus + '\t' + rsl + '\n')

3636165it [14:43, 4117.76it/s]
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:828: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


In [ ]:
#!g1.1


In [ ]:
#!g1.1
